# Introduçãos às CNNs no Keras e ilustrações dos resultados obtidos para cada camada da DNN

## Objetivos

Este notebook contém exemplo numérico de uma rede com uma camada convolucional e uma camada densa. A camada convolucional possui ativação reLU e max-pooling. Já a camada densa possui uma única saída com ativação sigmóide.

Com esse exemplo, aprende-se a:
- construir a rede utilizando tanto o modelo sequencial como o via API
- inicializar os pesos e biases da rede convolucional e da rede densa
- visualizar os dados intermediários da rede

## Importação dos Módulos

In [1]:
import numpy as np
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

import keras
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Input

np.set_printoptions(precision=3) # ponto flutuante com 3 casas para facilitar a impressão
print('Versão do Keras:', keras.__version__)

Using TensorFlow backend.


Versão do Keras: 2.0.3


In [2]:
import keras.backend as K

K.set_image_data_format('channels_first')
K.set_floatx('float32')

print('Backend:     {}'.format(K.backend()))
print('Data format: {}'.format(K.image_data_format()))


Backend:     tensorflow
Data format: channels_first


## Definição da Rede, camadas convolucionais e densas

Uma CNN pode ser construída de dois modos no Keras. Ela pode ser implementada usando

* Modelo Sequential: modelo mais simpes, utilizado quando a rede é composto de camadas uma após a outra.
* Modelo API: é o modelo mais geral, baseado em ligações de entradas e saídas, para construir um modelo 
  genérico contendo bifurcações e junções.
  

### Definição de camadas no Keras

Em redes neuras, uma camada é usualmente um neurônio, que inclui a soma de multiplicação de
pesos ou convolução e uma ativação usualmente não linear. Podemos dizer que o max-pooling 
também faz parte da camada.

A rede que iremos utilizar neste exemplo possui 2 camadas: uma convolucional e outra densa.
A camada convolucional terá ativação reLU e um max-pooling, já a camada densa terá uma
ativação sigmoide.

Para o Keras, entretanto, *layers* é um conceito diferente. Qualquer elemento que está
sendo construído no gráfico da rede é denominado *layer*. Assim no exemplo deste notebook,
existem sete *layers*: convolução, reLU, max-pooling, flat, dense e sigmoide.
 

### Formato channels_first ou channels_last

O Keras permite dois formatos de tensores: `channels_first` ou `channels_last`.
Neste treinamento, iremos utilizar quase sempre o formato `channels_first` por ser
mais intuitivo na formação dos tensores. É preciso cuidado, pois existe uma
configuração *default* que pode ser diferente da que você espera. Assim, uma boa
prática e deixar este parâmetro explícito nas chamadas das funções e métodos.

### Rede a ser implementada

<img src='../figures/RedeIntroKeras.png', width=600pt>

### Implementação pela API

In [3]:
def build_model_API():
    
    input_shape = (1,5,6) # (canais, linhas, colunas)
    n_filters = 3
    kernel_shape = (2,2)
    use_bias = True
    inputs = Input(input_shape, name = 'input')
    conv1 = Conv2D(n_filters, 
                   kernel_shape, 
                   name = 'conv1', 
                   padding = 'valid',
                   use_bias = use_bias,
                   data_format = "channels_first")(inputs) 
    actv1 = Activation('relu', name = 'relu1')(conv1)
    pool1 = MaxPooling2D(pool_size=(2,2),
                         data_format = "channels_first",
                         name = 'max_pool1')(actv1) 
    flat = Flatten(name = 'flat')(pool1)
    dense1 = Dense(1, 
                   use_bias = use_bias,
                   name = 'dense2')(flat)
    out = Activation('sigmoid', name = 'sigmoid2')(dense1)

    model = Model(inputs=inputs, outputs=out)
    
    return model

### Implementação pelo modelo *Sequencial*

In [4]:
def build_model_Sequential():
       
    input_shape = (1,5,6) # (canais, linhas, colunas)
    n_filters = 3
    kernel_shape = (2,2)
    use_bias = True
    model = Sequential()
    
    model.add(Conv2D(n_filters, # número de filtros
                     kernel_shape, # tamanho do kernel
                     name = 'conv_1', 
                     padding = 'valid',
                     use_bias = use_bias,
                     input_shape = input_shape,
                     data_format = "channels_first"))
    model.add(Activation('relu', name = 'relu_1'))
    model.add(MaxPooling2D(pool_size = (2, 2), 
                           data_format = "channels_first",
                           name = 'max_pool_1'))
    model.add(Flatten(name = 'flat'))
    model.add(Dense(1, 
                    use_bias = use_bias,
                    name = 'dense_2'))
    model.add(Activation('sigmoid', name = 'sigmoid_2'))
    
    return model

# Configurando a rede neural com convolução e densa

## Definição dos parâmetros da rede

### Imagem de Entrada: 5 linhas e 6 colunas

In [5]:
X = np.array([[[[0,0,0,0,0,0],
                [0,0,1,0,0,0],
                [0,0,0,0,0,0],
                [0,0,0,0,-1,0],
                [0,0,0,0,0,0]]]])

(n_samples, n_channels, img_height, img_width) = X.shape
input_shape = (n_channels, img_height, img_width)
print('X.shape=',X.shape)
print('X:\n',X)

X.shape= (1, 1, 5, 6)
X:
 [[[[ 0  0  0  0  0  0]
   [ 0  0  1  0  0  0]
   [ 0  0  0  0  0  0]
   [ 0  0  0  0 -1  0]
   [ 0  0  0  0  0  0]]]]


### Kernel da convolução

In [6]:
# número de filtros
n_filters = 3    

# comprimento e largura dos filtros
k_height = k_width = 2 
kernel_shape = (k_height,k_width)

Win = np.array([[[1,2],
                 [3,4]],
                [[5,6],
                 [7,8]],
                [[9,10],
                 [11,12]]]).reshape(n_filters,1,k_height,k_width)

print('Win.shape (n_filters, n_channels, k_height, k_width):',Win.shape)
print('Win: (Pesos camada convolucional, formato mais fácil de entender)\n',Win)

W_conv = Win.transpose(2,3,1,0)
# Para deixar no formato do Keras: (k_height,k_width,n_channels,n_filters)
print('Wconv.shape (k_height,k_width,n_channels,n_filters):',W_conv.shape)
print('Wconv: (Pesos camada convolucional, formato do Keras)\n',W_conv)


Win.shape (n_filters, n_channels, k_height, k_width): (3, 1, 2, 2)
Win: (Pesos camada convolucional, formato mais fácil de entender)
 [[[[ 1  2]
   [ 3  4]]]


 [[[ 5  6]
   [ 7  8]]]


 [[[ 9 10]
   [11 12]]]]
Wconv.shape (k_height,k_width,n_channels,n_filters): (2, 2, 1, 3)
Wconv: (Pesos camada convolucional, formato do Keras)
 [[[[ 1  5  9]]

  [[ 2  6 10]]]


 [[[ 3  7 11]]

  [[ 4  8 12]]]]


### Bias da Convolução

In [7]:
# valor de bias
f_bias = 0.1     
bias_conv = np.arange(1,n_filters+1) * f_bias
print("Bias da convolução:",bias_conv)

Bias da convolução: [ 0.1  0.2  0.3]


### Pesos para a camada densa

In [8]:
# após o max pooling, são 3 imagens 2x2 = 12
W_dense = np.arange(12).reshape(12,1)
print("Pesos da camada densa:\n",W_dense)

Pesos da camada densa:
 [[ 0]
 [ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [10]
 [11]]


### Bias para a camada densa

In [9]:
bias_dense = np.ones(1) * f_bias
print("Bias da camanda densa:",bias_dense)

Bias da camanda densa: [ 0.1]


## Imprimindo os valores dos dados das camadas internas da rede

Como a rede é executada no Keras como *backend*, não existem variáveis de fácil acesso para
conseguir ver os dados nas camadas intermediárias da rede. Assim, o artifício para se conseguir
isso é criar várias redes, uma rede com apenas a primeira camada, e fazer a predição. O resultado
dessa predição são os dados após a primeira camada. Colocar a segunda camada e fazer uma nova predição, obtendo
os dados após a segunda camada e assim sucessivamente até fazer a predição da rede completa. Isso
é implementado no código a seguir:

In [10]:
model = build_model_API()
#model = build_model_Sequential()
print(W_conv.shape)
print(bias_conv.shape)
print(W_dense.shape)
print(bias_dense.shape)
model.set_weights([W_conv, bias_conv, W_dense, bias_dense])

print('-'*30)
print("Número de camadas:", len(model.layers))
print('-'*30)

# Resultados para cada camada
i = 1
for layer in model.layers:
    intermediate_layer_model = Model(inputs=model.input,outputs=layer.output)
    intermediate_output = intermediate_layer_model.predict(X)
    print('-'*80)
    print("Saída da camada", i, ":", layer.name, "shape:", intermediate_output.shape)
    print('-'*80)
    print(intermediate_output)
    i+=1

(2, 2, 1, 3)
(3,)
(12, 1)
(1,)
------------------------------
Número de camadas: 7
------------------------------
--------------------------------------------------------------------------------
Saída da camada 1 : input shape: (1, 1, 5, 6)
--------------------------------------------------------------------------------
[[[[ 0.  0.  0.  0.  0.  0.]
   [ 0.  0.  1.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0. -1.  0.]
   [ 0.  0.  0.  0.  0.  0.]]]]
--------------------------------------------------------------------------------
Saída da camada 2 : conv1 shape: (1, 3, 4, 5)
--------------------------------------------------------------------------------
[[[[  0.1   4.1   3.1   0.1   0.1]
   [  0.1   2.1   1.1   0.1   0.1]
   [  0.1   0.1   0.1  -3.9  -2.9]
   [  0.1   0.1   0.1  -1.9  -0.9]]

  [[  0.2   8.2   7.2   0.2   0.2]
   [  0.2   6.2   5.2   0.2   0.2]
   [  0.2   0.2   0.2  -7.8  -6.8]
   [  0.2   0.2   0.2  -5.8  -4.8]]

  [[  0.3  12.3  11.3   0.3   0.3]
   

## Sumário 

É sempre útil imprimir o sumário da rede. Ele contém informações de cada camada da rede, 
como nome, tipo de camada, shape do tensor na saída da camada e número de parâmetros.

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 1, 5, 6)           0         
_________________________________________________________________
conv1 (Conv2D)               (None, 3, 4, 5)           15        
_________________________________________________________________
relu1 (Activation)           (None, 3, 4, 5)           0         
_________________________________________________________________
max_pool1 (MaxPooling2D)     (None, 3, 2, 2)           0         
_________________________________________________________________
flat (Flatten)               (None, 12)                0         
_________________________________________________________________
dense2 (Dense)               (None, 1)                 13        
_________________________________________________________________
sigmoid2 (Activation)        (None, 1)                 0         
Total para

# Sugestões de atividades

1. Executar o experimento nos dois modos de construção da rede, API e Sequential. 
   Existe alguma diferença?
2. Calcular o número de parâmetros da rede. Confirmar com o sumário apresentado.
3. Retirar o bias e recalcular o número de parâmetros a serem treinados
4. A rede foi projetada para aceitar entrada com shape (1,5,6). Mudar a rede para aceitar:
   a) entrada com shape (1,6,6)
   b) entrada com shape (3,6,6)
5. Inserir mais uma camada convolucional com 4 filtros de saída e kernel (3,3)


# Referências

- [Keras Documentation](https://keras.io/)


# Aprendizados com este notebook
